# Dashboard

In [2]:
# export
import os
import re
import numpy as np
import pandas as pd
import panel as pn
import webbrowser
pn.extension('plotly')

import pyteomics.fasta
from io import StringIO
import plotly.graph_objects as go

from pepmap.importing import import_data
from pepmap.preprocessing import format_input_data
from pepmap.sequenceplot import plot_peptide_traces
from pepmap.uniprot_integration import uniprot_feature_dict
from pepmap.sequenceplot import uniprot_color_dict

In [3]:
path_to_folder_fasta_files = 'data'
full_fasta = None
full_uniprot = None
ac_gene_conversion = None

### Style of the dashboard

In [4]:
#export
css = '''

.bk-root .bk-btn-default.bk-active {
    background-color: #AECCE6;
}

.bk.accordion {
    font-size: smaller;
}

.bk-root .bk-btn {
    padding: 6px 0px 6px 0px;
}

.bk.uniprot_options {
    margin: 17px 6px 12px 6px;
    background-color: white;
    font-size: 13px;
    font-weight: initial;
    text-align: center;
    height: 55px;
    border: 1px solid #e0e0e0;
    border-radius: 2px;
}

.bk-root .bk-btn-primary {
    background-color: #045082;
    font-size: 11px;
    font-weight: 700;
    text-transform: uppercase;
    letter-spacing: 1.5px;
}

.bk.card-title {
    font-size: 13px;
    font-weight: initial;
}

h1 {
    color: #045082;
    font-size: 45px;
    line-height: 0.6;
    text-align: center;
}

h4 {
    color: #173245;
    font-size: 18px;
    line-height: 0.6;
    text-align: center;
}

.codehilite {
    color: #173245;
    font-size: 14px;
    background-color: #eaeaea;
}

.codehilite .s2 {
    color: #173245;
    font-weight: bold;
}

.codehilite .ow {
    color: #045082;
    font-weight: initial;
}

.bk.opt {
    position: relative;
    display: block;
    left: 75px;
    top: 0px;
    width: 80px;
    height: 80px;
}

.bk.main-part {
    background-color: #EAEAEA;
    font-size: 17px;
    line-height: 23px;
    letter-spacing: 0px;
    font-weight: 500;
    color: #045082;
    text-align: center;
    position: relative !important;
    margin-left: auto;
    margin-right: auto;
    width: 40%;
}

.bk.selection_box {
    background-color: #EAEAEA;
}

.bk.alert-danger {
    background-color: #EAEAEA;
    color: #c72d3b;
    border: 0px #EAEAEA solid;
    padding: 0;
    font-size: 12px;
}

'''
pn.extension(raw_css=[css])

### Header

In [5]:
### HEADER
header_titel = pn.pane.Markdown(
    '# AlphaMap', 
    width=1250,
    css_classes=['main_header']
)
divider = pn.pane.HTML(
    '<hr style="height: 2px; border:none; background-color: #045082; width: 1480px">', 
    width=1500, 
    align='center'
)
mpi_biochem_logo = pn.pane.PNG(
    'data/img/mpi_logo.png', 
    link_url='https://www.biochem.mpg.de/en', 
    width=60, 
    height=60,
    align='start'
)
mpi_logo = pn.pane.JPG(
    'data/img/max-planck-gesellschaft.jpg', 
    link_url='https://www.biochem.mpg.de/en',
    height=62,
    embed=True,
    width=62,
    margin=(5, 0, 0, 5),
    css_classes=['opt']
)
header = pn.Row(
    mpi_biochem_logo,
    mpi_logo,
    header_titel,
    height=70
)

In [6]:
### WIDGETS

#####################################
# SELECTORS
select_protein = pn.widgets.AutocompleteInput(
    name='Select a Uniprot accession number:', 
    placeholder='Type first letters ...',
    min_characters=1
)
select_organism = pn.widgets.Select(
    name='Select an organism:', 
    value='Human', 
    options=['Human', 'Mouse', 'Arabidopsis'],
    align='center',
    margin=(0,0,0,7),
    width=300
)

#####################################
# CHECKBOXES
is_gene = pn.widgets.Checkbox(
    name='Search by a gene name'
)
select_all = pn.widgets.Checkbox(
    name='Select all options', 
    width=150
)
clear_all = pn.widgets.Checkbox(
    name='Clear all options',
    width=150
)

#####################################
# TEXT
upload_file_text = pn.pane.Markdown(
    'Upload an experimental file:',
    margin=(10,0,-5,15)
)
upload_file_text_inside_card = pn.pane.Markdown(
    'Upload an experimental file:',
    margin=(5,0,-5,15)
)
#####################################
# RAW EXPERIMENTAL DATA

error_message = "The selected file can't be uploaded. Please check the instructions for data uploading."

# first experimental file
experimental_data = pn.widgets.FileInput(
    accept=".csv, .txt, .tsv", 
    width=400,
    margin=(-5,-30,5,15)
)
experimental_data_sample = pn.widgets.MultiSelect(
    name='Select samples:', 
    disabled=True,
    size=5,
    width=620,
    margin=(0,0,20,14)
)
experimental_data_warning = pn.pane.Alert(
    width=550,
    height=30,
    alert_type="danger", 
    margin=(-20,0,15,15)
)
experimental_data_sample_name = pn.widgets.TextInput(
    name='Rename samples:', 
    disabled=True,
    width=240,
    margin=(23,0,5,10)
)
# second experimental file
experimental_data_2 = pn.widgets.FileInput(
    accept=".csv, .txt, .tsv", 
    width=400,
    margin=(-5,-30,5,15)
)
experimental_data_2_sample = pn.widgets.MultiSelect(
    name='Select samples:', 
    size=5,
    disabled=True,
    width=620,
    margin=(0,0,0,14)
)
experimental_data_2_warning = pn.pane.Alert(
    width=550,
    height=30,
    alert_type="danger", 
    margin=(-20,0,15,15)
)
experimental_data_2_sample_name = pn.widgets.TextInput(
    name='Rename samples:', 
    disabled=True,
    width=240,
    margin=(18,0,5,10)
)
# third experimental file
experimental_data_3 = pn.widgets.FileInput(
    accept=".csv, .txt, .tsv", 
    width=400,
    margin=(-5,-30,5,15)
)
experimental_data_3_sample = pn.widgets.MultiSelect(
    name='Select samples:', 
    size=5,
    disabled=True,
    width=620,
    margin=(0,0,20,14)
)
experimental_data_3_warning = pn.pane.Alert(
    width=550,
    height=30,
    alert_type="danger", 
    margin=(-20,0,15,15)
)
experimental_data_3_sample_name = pn.widgets.TextInput(
    name='Rename samples:', 
    disabled=True,
    width=240,
    margin=(18,0,5,10)
)

#####################################
# PREPROCESSED EXPERIMENTAL DATA
preprocessed_exp_data = pn.widgets.DataFrame(
    name='Exp_data'
)
preprocessed_exp_data_2 = pn.widgets.DataFrame(
    name='Exp_data_2'
)
preprocessed_exp_data_3 = pn.widgets.DataFrame(
    name='Exp_data_3'
)

#####################################
# BUTTONS
upload_button = pn.widgets.Button(
    name='Upload  Data', 
    button_type='primary', 
    css_classes=['button_options'],
    height=40,
    width=150,
    align='center',
    margin=(0,0,0,0)
)
upload_spinner = pn.indicators.LoadingSpinner(
    value=False,
    bgcolor='light',
    color='secondary',
    align='center',
    margin=(0,0,20,20),
    width=40, 
    height=40
)
visualize_button = pn.widgets.Button(
    name='Visualize  Data', 
    button_type='primary', 
    css_classes=['button_options'],
    height=40,
    width=150,
    align='center',
    margin=(0,0,0,0)
)
visualize_spinner = pn.indicators.LoadingSpinner(
    value=False,
    bgcolor='light',
    color='secondary',
    align='center',
    margin=(0,0,20,20),
    width=40, 
    height=40
)

In [7]:
### UNIPROT OPTIONS
options_preprocessing_events = pn.widgets.CheckButtonGroup(
    name='Molecule processing', 
    value=['Chain', 'Initiator methionine', 'Peptide', 'Propeptide', 'Signal peptide', 'Transit peptide'], 
    options=['Chain', 'Initiator methionine', 'Peptide', 'Propeptide', 'Signal peptide', 'Transit peptide'],
    align='center'
)
options_PTMs = pn.widgets.CheckButtonGroup(
    name='Post-translational modification', 
    options=['Cross-link', 'Disulfide bond', 'Glycosylation', 'Lipidation', 'Modified residue'],
    value=['Cross-link', 'Disulfide bond', 'Glycosylation', 'Lipidation', 'Modified residue'],
    align='center'
)
options_domains = pn.widgets.CheckButtonGroup(
    name='Family & Domain',  
    options=['Coiled coil', 'Compositional bias', 'Domain', 'Motif', 'Region', 'Repeat', 'Zinc finger'],
    value=['Coiled coil', 'Compositional bias', 'Domain', 'Motif', 'Region', 'Repeat', 'Zinc finger'],
    align='center'
)
options_locations = pn.widgets.CheckButtonGroup(
    name='Subcellular location', 
    options=['Intramembrane', 'Topological domain', 'Transmembrane'],
    value=['Intramembrane', 'Topological domain', 'Transmembrane'],
    align='center'
)
options_functions = pn.widgets.CheckButtonGroup(
    name='Function', 
    options=['Active site', 'Binding site', 'Calcium binding', 'DNA binding', 'Metal binding', 'Nucleotide binding', 'Site'],
    value=['Active site', 'Binding site', 'Calcium binding', 'DNA binding', 'Metal binding', 'Nucleotide binding', 'Site'],
    align='center'
)
options_sequences = pn.widgets.CheckButtonGroup(
    name='Sequence', 
    options=['Alternative sequence', 'Natural variant', 'Non-adjacent residues', 'Non-standard residue', 
             'Non-terminal residue', 'Sequence conflict', 'Sequence uncertainty'],
    value=['Alternative sequence', 'Natural variant', 'Non-adjacent residues', 'Non-standard residue', 
             'Non-terminal residue', 'Sequence conflict', 'Sequence uncertainty'],
    align='center'
)
options_other = pn.widgets.CheckButtonGroup(
    name='Other options', 
    options=['Secondary structure', 'Mutagenesis'],
    value=['Secondary structure', 'Mutagenesis'],
    align='center'
)

#####################################
### LAYOUTS
uniprot_options = pn.Accordion(
    options_preprocessing_events, 
    options_PTMs,
    options_domains,
    options_locations,
    options_functions,
    options_sequences,
    options_other,
    active=list(range(0,7)),
    header_background='EAEAEA',
    active_header_background='EAEAEA',
    width = 850
)

uniprot_options_tab = pn.Card(
    uniprot_options,
    pn.Row(
        select_all,
        clear_all, 
        margin=5
    ),
    title='Uniprot Options',
    collapsed=True,
    header_background='EAEAEA',
    active_header_background='EAEAEA',
    width=860,
    css_classes=['uniprot_options'],
)

In [8]:
### MAIN PART
project_description = pn.pane.Markdown(
    """### This website enables the exploration of proteomic datasets on the peptide level. You can evaluate the sequence coverage of any selected protein and its identified post-translational modifications (PTMs). UniProt information is available to directly annotate sequence regions of interest such as protein domains, secondary structures, sequence variants, known PTMs, etc. """,
    margin=(10, 0, -20, 0), 
    css_classes=['main-part'], 
    width=615
)

divider_descr = pn.pane.HTML(
    '<hr style="height: 8px; border:none; background-color: #045082; width: 620px">', 
    width=1510, 
    align='center'
)

project_instuction = pn.pane.Markdown(
    """#### How to use AlphaMap:
    1. Select the organism of your proteomic study.  
    2. Select up to 3 different bottom-up proteomic datasets analyzed  
    by AlphaPept, MaxQuant or Spectronaut.  
    3. Press the "Upload Data" button.  
    4. Select a protein of interest.  
    5. Select desired UniProt sequence annotations.  
    6. Press the "Visualize Data" button.  
    7. Enjoy exploring your data!  
    """,
    width=500,
    align='start',
    margin=(25, 80, 0, 10)
)

spectronaut_description = pn.pane.Markdown(
    """
    The data needs to be exported in the **normal long** format as .tsv or .csv file. 
    
    It needs to include the following columns:  
    >- PEP.AllOccurringProteinAccessions  
    >- EG.ModifiedSequence  
    >- R.FileName  
    
    To ensure the correct export format from Spectronaut, you can download and  
    apply the provided export scheme “spectronaut_export_scheme.rs”.
    """,
    width=500,
    align='start',
    margin=(0, 80, 0, 20)
)

spectronaut_scheme = pn.widgets.FileDownload(
    file='data/spectronaut_export_scheme.rs',
    filename='spectronaut_export_scheme.rs',
    button_type='default', 
    auto=True,
    css_classes=['button_options'],
)

maxquant_description = pn.pane.Markdown(
    """
    To visualize the proteins which were analyzed by the MaxQuant software please use the **evidence.txt** file.

    The following columns from the file are used for visualization:
    >- Proteins
    >- Modified sequence
    >- Raw file
    """,
    width=500,
    align='start',
    margin=(0, 80, 0, 20)
)

spectronaut_instructions = pn.Card(
    spectronaut_description,
    spectronaut_scheme,
    title='Spectronaut instructions',
    collapsed=True,
    width=505,
    align='start',
    margin=(10, 80, 5, 10),
    css_classes=['spectronaut_instr']
)

maxquant_instructions = pn.Card(
    maxquant_description,
    title='MaxQuant instructions',
    collapsed=True,
    width=505,
    align='start',
    margin=(0, 80, 5, 10),
    css_classes=['spectronaut_instr']
)

additional_data_card = pn.Card(
    pn.Row(
        pn.Column(
            upload_file_text_inside_card,
            experimental_data_2
        ),
        experimental_data_2_sample_name
    ),
    experimental_data_2_warning,
    experimental_data_2_sample,
    pn.Row(
        pn.Column(
            upload_file_text_inside_card,
            experimental_data_3
        ),
        experimental_data_3_sample_name
    ),
    experimental_data_3_warning,
    experimental_data_3_sample,
    title='Upload additional experimental files',
    collapsed=True,
    width=640,
    margin=(2,0,0,5),
    css_classes=['add_experim_options']
)

selection_box = pn.Column(
    select_organism,
    pn.Row(
        pn.Column(
            upload_file_text,
            experimental_data
        ),
        experimental_data_sample_name
    ),
    experimental_data_warning,
    experimental_data_sample,
    additional_data_card,
    margin=(20, 150, 10, 80),
    width=550,
    css_classes=['selection_box'],
)

main_part = pn.Column(
    project_description,
    divider_descr,
    pn.Row(
        pn.Column(
            project_instuction,
            spectronaut_instructions, 
            maxquant_instructions
        ),
        selection_box,
        align='center'
    ),
    pn.Row(
        upload_button,
        upload_spinner,
        align='center'
    ),
    background='#eaeaea',
    width=1510,
    margin=(5, 0, 10, 0)
)

In [10]:
# switch to different websites
uniprot_link = pn.pane.PNG(
    'data/img/uniprot_logo.png', 
    width=120, 
    height=60,
    align='start',
    margin=(0, 30, 0, 40)
)
phosposite_link = pn.pane.PNG(
    'data/img/phosphosite_logo.png',  
    width=200, 
    height=60,
    align='start',
    margin=(0, 20)
)
protter_link = pn.pane.PNG(
    'data/img/protter_logo.png', 
    width=140, 
    height=60,
    align='start',
    margin=(0, 20)
)
pdb_link = pn.pane.PNG(
    'data/img/pdb_logo.png', 
    width=120, 
    height=60,
    align='start',
    margin=(0, 20)
)

def extract_uniprot_ai(protein, is_gene):
    if is_gene:
        return re.findall(r"\((?P<id>.+?)\)", protein)[0]
    return protein
        

def update_all_links():
    selected_protein_id = extract_uniprot_ai(select_protein.value, is_gene.value)
    uniprot_link.link_url = 'https://www.uniprot.org/uniprot/' + selected_protein_id
    phosposite_link.link_url = 'http://www.phosphosite.org/uniprotAccAction?id=' + selected_protein_id
    protter_link.link_url = 'https://wlab.ethz.ch/protter/#up=' + selected_protein_id
    pdb_link.link_url = f'https://www.rcsb.org/search?request=%7B%22query%22%3A%7B%22parameters%22%3A%7B%22value%22%3A%22{selected_protein_id}%22%7D%2C%22type%22%3A%22terminal%22%2C%22service%22%3A%22text%22%2C%22node_id%22%3A0%7D%2C%22return_type%22%3A%22entry%22%2C%22request_options%22%3A%7B%22pager%22%3A%7B%22start%22%3A0%2C%22rows%22%3A100%7D%2C%22scoring_strategy%22%3A%22combined%22%2C%22sort%22%3A%5B%7B%22sort_by%22%3A%22score%22%2C%22direction%22%3A%22desc%22%7D%5D%7D%2C%22request_info%22%3A%7B%22src%22%3A%22ui%22%2C%22query_id%22%3A%223407f72e3370cd10196490437be3ec87%22%7D%7D'
    

@pn.depends(visualize_button.param.clicks)
def visualize_buttons(_):
    if select_protein.value:
        update_all_links()
        buttons_layout = pn.Row(
            pn.pane.Markdown(
                "### Switch to the following websites:", 
                margin=(16, 0, 0, 0)
            ),
            uniprot_link,
            phosposite_link,
            protter_link,
            pdb_link,
            height=60,
            align='start',
            margin=(0, 0, 0, 210)
        )
        return buttons_layout
    else:
        return None


In [ ]:
### PREPROCESSING
def upload_experimental_data():
    global ac_gene_conversion
    if experimental_data_sample.value == ['All samples']:
        data_samples = None
    else:
        data_samples = experimental_data_sample.value
    preprocessed_exp_data.value = format_input_data(
        df = import_data(
            experimental_data.value, 
            dashboard=True, 
            verbose=False, 
            sample=data_samples
        ), 
        fasta = full_fasta, 
        modification_exp = r'\[.*?\]', 
        verbose = False)
    all_unique_proteins = preprocessed_exp_data.value.unique_protein_id.unique().tolist()
    if experimental_data_2.filename:
        if experimental_data_2_sample.value == ['All samples']:
            data_2_samples = None
        else:
            data_2_samples = experimental_data_2_sample.value
        try:
            preprocessed_exp_data_2.value = format_input_data(
                df = import_data(
                    experimental_data_2.value, 
                    dashboard=True, 
                    verbose=False,
                    sample=data_2_samples
                ), 
                fasta = full_fasta, 
                modification_exp = r'\[.*?\]', 
                verbose = False)
            all_unique_proteins.extend(preprocessed_exp_data_2.value.unique_protein_id.unique().tolist())
        except (TypeError, AttributeError):
            pass
#             print(error_message)
    if experimental_data_3.filename:
        if experimental_data_3_sample.value == ['All samples']:
            data_3_samples = None
        else:
            data_3_samples = experimental_data_3_sample.value
        try:
            preprocessed_exp_data_3.value = format_input_data(
                df = import_data(
                    experimental_data_3.value, 
                    dashboard=True, 
                    verbose=False,
                    sample=data_3_samples
                ), 
                fasta = full_fasta, 
                modification_exp = r'\[.*?\]', 
                verbose = False)
            all_unique_proteins.extend(preprocessed_exp_data_3.value.unique_protein_id.unique().tolist())
        except (TypeError, AttributeError):
#             print(error_message)
            pass
    ac_gene_conversion = {
        each: f"{full_fasta.get_by_id(each).description.get('GN')} ({full_fasta.get_by_id(each).description.get('id')})" \
        for each in sorted(list(set(all_unique_proteins)))}
    # to set a selection list of availible proteins depending whether a is_gene checkbox is checked
    if is_gene.value:
        select_protein.options = list(ac_gene_conversion.values())
    else:
        select_protein.options = list(ac_gene_conversion.keys())
    
    
def upload_organism_info():
    global full_fasta, full_uniprot
    if select_organism.value == 'Human':
        fasta_name = "human.fasta"
        uniprot_name = "preprocessed_uniprot_human.csv"
    elif select_organism.value == 'Mouse':
        fasta_name = "mouse.fasta"
        uniprot_name = "preprocessed_uniprot_mouse.csv"
    elif select_organism.value == 'Arabidopsis':
        fasta_name = "arabidopsis_thaliana.fasta"
        uniprot_name = "preprocessed_uniprot_arabidopsis.csv"
    full_fasta = pyteomics.fasta.IndexedUniProt(os.path.join(path_to_folder_fasta_files, fasta_name))
    full_uniprot = pd.read_csv(os.path.join(path_to_folder_fasta_files, uniprot_name))
    

def extract_samples(data):
    """
    Extract information about unique sample names that present in the raw file analyzed by MaxQuant or Spectronaut.
    """
    sample_columns = ['R.FileName', 'Raw file']
    try:
        unique_samples = pd.read_csv(StringIO(str(data, "utf-8")), sep=None, engine='python', 
            usecols=lambda col: col in sample_columns).iloc[:, 0].unique().tolist()
    except:
        raise TypeError("This file can't be uploaded.")
    return unique_samples

def extract_name(filename, sample, sample_name):
    if sample_name:
        name = sample_name
    elif sample and sample != ['All samples']:
        if isinstance(sample, list):
            name = ";".join(sample)
        else:
            name = sample
    else:
        name = filename.split('.')[0]
    return name

@pn.depends(
    experimental_data.param.value, 
    watch=True
)
def update_data_sample_info(data1):
    experimental_data_sample.disabled = False
    try:
        experimental_data_warning.object = None
        experimental_data_sample.options = ['All samples'] + extract_samples(data1) 
        experimental_data_sample.value = ['All samples']
    except TypeError:
        experimental_data_warning.object = error_message
        experimental_data_sample.disabled = True
        experimental_data_sample_name.disabled = True
        experimental_data_sample.options = []
        experimental_data_sample.value = []
        experimental_data_sample_name.value = ''

@pn.depends(
    experimental_data_2.param.value,
    watch=True
)
def update_data_2_sample_info(data2):
    experimental_data_2_sample.disabled = False
    try:
        experimental_data_2_warning.object = None
        experimental_data_2_sample.options = ['All samples'] + extract_samples(data2) 
        experimental_data_2_sample.value = ['All samples']
    except TypeError:
        experimental_data_2_warning.object = error_message
        experimental_data_2_sample.disabled = True
        experimental_data_2_sample_name.disabled = True
        experimental_data_2_sample.options = []
        experimental_data_2_sample.value = []
        experimental_data_2_sample_name.value = ''
    
@pn.depends(
    experimental_data_3.param.value,
    watch=True
)
def update_data_3_sample_info(data3):
    experimental_data_3_sample.disabled = False
    try:
        experimental_data_3_warning.object = None
        experimental_data_3_sample.options = ['All samples'] + extract_samples(data3) 
        experimental_data_3_sample.value = ['All samples']
    except TypeError:
        experimental_data_3_warning.object = error_message
        experimental_data_3_sample.disabled = True
        experimental_data_3_sample_name.disabled = True
        experimental_data_3_sample.options = []
        experimental_data_3_sample.value = []
        experimental_data_3_sample_name.value = ''
    
    
@pn.depends(
    experimental_data_sample.param.value,
    experimental_data_2_sample.param.value,
    experimental_data_3_sample.param.value,
    watch=True
)
def change_sample_name_state(data_sample, data_2_sample, data_3_sample):
    if data_sample:
        experimental_data_sample_name.disabled = False
    else:
        experimental_data_sample_name.disabled = True
    if data_2_sample:
        experimental_data_2_sample_name.disabled = False
    else:
        experimental_data_2_sample_name.disabled = True
    if data_3_sample:
        experimental_data_3_sample_name.disabled = False 
    else:
        experimental_data_3_sample_name.disabled = True

@pn.depends(
    select_all.param.value,
    clear_all.param.value,
    watch=True
)
def change_uniprot_selection(select, clear):
    if select:
        clear_all.value = False
        select_all.value = False
        for each in uniprot_options.objects:
            each.value = each.options
    if clear:
        clear_all.value = False
        select_all.value = False
        for each in uniprot_options.objects:
            each.value = []
        
        
@pn.depends(
    is_gene.param.value, 
    watch=True
)
def change_autocomplete_input(is_gene_selected):
    if any(
        [experimental_data_sample.value, experimental_data_2_sample.value, experimental_data_3_sample.value]
    ):
        if is_gene_selected:
            select_protein.options = list(ac_gene_conversion.values())
        else:
            select_protein.options = list(ac_gene_conversion.keys())


In [ ]:
### VISUALIZATION
@pn.depends(
    upload_button.param.clicks
)
def upload_data(clicks):
    if clicks > 0 and any(
        [experimental_data_sample.value, experimental_data_2_sample.value, experimental_data_3_sample.value]
    ):
        upload_spinner.value = True
        select_protein.value = None
        # preload the data
        upload_organism_info()
        upload_experimental_data()
        # create a layout
        app = pn.Column(
            pn.Row(
                pn.layout.VSpacer(width=100),
                pn.Column(
                    select_protein,
                    is_gene
                ),
                pn.layout.VSpacer(width=80),
                uniprot_options_tab,
                align='center'
            ),
            pn.layout.HSpacer(height=4),
            pn.Row(
                visualize_button,
                visualize_spinner,
                align='center'
            ),
            divider,
            pn.Pane(
                visualize_plot, 
                config={'toImageButtonOptions': 
                           {'format': 'svg', # one of png, svg, jpeg, webp
                            'filename': 'custom_image',
                            'height': 500,
                            'width': 1500,
                            'scale': 1 # Multiply title/legend/axis/canvas sizes by this factor
                           }
                       }, 
                align='center',
                width=1500
            ),
            visualize_buttons,
            margin=(20, 0)
        )
        upload_spinner.value = False
        return app
    
    
@pn.depends(visualize_button.param.clicks)
def visualize_plot(_):
    if select_protein.value:
        visualize_spinner.value = True
        # combine selected uniprot options in one list
        uniprot_options_combined = sum([each.value for each in uniprot_options.objects if each.value], [])
        # extract all experimental data and names 
        all_data = []
        all_names = []
        if preprocessed_exp_data.value is not None:
            all_data.append(preprocessed_exp_data.value)
            all_names.append(
                extract_name(experimental_data.filename, 
                             experimental_data_sample.value, 
                             experimental_data_sample_name.value
                )
            )
        if preprocessed_exp_data_2.value is not None:
            all_data.append(preprocessed_exp_data_2.value)
            all_names.append(
                extract_name(experimental_data_2.filename, 
                             experimental_data_2_sample.value, 
                             experimental_data_2_sample_name.value
                )
            )
        if preprocessed_exp_data_3.value is not None:
            all_data.append(preprocessed_exp_data_3.value)
            all_names.append(
                extract_name(experimental_data_3.filename, 
                             experimental_data_3_sample.value, 
                             experimental_data_3_sample_name.value
                )
            )
        # if only one experimental file is uploaded we need to return a string for input into plot_peptide_traces
        if len(all_data) == 1:
            all_data = all_data[0]
            all_names = all_names[0]
        if is_gene.value:
            selected_protein = re.findall(r"\((?P<id>.+?)\)", select_protein.value)[0]
        else:
            selected_protein = select_protein.value
        # create a main figure
        fig =  plot_peptide_traces(
            df = all_data,
            name = all_names,
            protein = selected_protein,
            fasta = full_fasta, 
            uniprot = full_uniprot,
            selected_features = [uniprot_feature_dict[each] for each in uniprot_options_combined],
            uniprot_feature_dict = uniprot_feature_dict, 
            uniprot_color_dict = uniprot_color_dict)
        visualize_spinner.value = False
        return fig
    else:
        return None

In [ ]:
### JS callbacks to control the behaviour of pn.Cards
uniprot_options_tab.jscallback(
    collapsed=""" 
        var $container = $("html,body");
        var $scrollTo = $('.uniprot_options');

        $container.animate({scrollTop: $container.offset().top + $container.scrollTop(), scrollLeft: 0},300);
        """, 
    args={'card': uniprot_options_tab}
);

additional_data_card.jscallback(
    collapsed=""" 
        var $container = $("html,body");
        var $scrollTo = $('.add_experim_options');

        $container.animate({scrollTop: $container.offset().top + $container.scrollTop(), scrollLeft: 0},300);
        """, 
    args={'card': additional_data_card}
);

spectronaut_instructions.jscallback(
    collapsed=""" 
        var $container = $("html,body");
        var $scrollTo = $('.spectronaut_instr');

        $container.animate({scrollTop: $container.offset().top + $container.scrollTop(), scrollLeft: 0},300);
        """, 
    args={'card': spectronaut_instructions}
);

maxquant_instructions.jscallback(
    collapsed=""" 
        var $container = $("html,body");
        var $scrollTo = $('.spectronaut_instr');

        $container.animate({scrollTop: $container.offset().top + $container.scrollTop(), scrollLeft: 0},300);
        """, 
    args={'card': maxquant_instructions}
);

In [ ]:
layout = pn.Column(
    header,
    main_part,
    upload_data
)

In [ ]:
layout.servable()